## Table of Contents
1. [Unzip](#unzipping): **start here if first time running locally** unzipping historic data, converting and exporting as `.csv`   
1. [Import](#csv-imports): **start here if `.csv` files are locally available** Importing data as `.csv` files   
1. [Initial cleaning](#initial-cleaning): filling nulls, removing deleted records and duplicates, exporting `.csv` files    
1. [Datatype management](#datatype-management): converting datatypes, setting up map for imports   

In [1]:
import os 
import io
import json
from datetime import datetime
import numpy as np
import pandas as pd
import zstandard as zstd
from cleaning_assets import *

In [2]:
df = pd.read_csv('./Data/stats/comments.csv')

<a class="anchor" name="unzipping"></a>

### Unzipping & transforming original files from [The Eye](https://the-eye.eu/redarcs/)
#### `.zst` file processing & `.csv` transformation
Skip to [next section](#csv-imports) if `.csv` files are locally available.

In [2]:
# Long run time - only execute if starting from beginning, otherwise skip to import csv
# Reading in files
for file in os.listdir('./Data/original'):
    file_name = file[:-4].lower()
    site_name = file.split('_')[0].lower()
    
    # unzipper put together by Tamir Cohen
    with open(f'./Data/original/{file}', 'rb') as fh:
            # decompress
        dctx = zstd.ZstdDecompressor(max_window_size=2_147_483_648)
        stream_reader = dctx.stream_reader(fh)
        text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8')
        data = [] 
            # parse JSONs & add rows
        for line in text_stream:
            data.append(json.loads(line))
        # store to correct container
    if 'comments' in file_name:
        all_comments[site_name] = pd.DataFrame(data)
        print(f'{file_name} unzipped, containing {len(all_posts[site_name])} records')
    elif 'submissions' in file_name:
        all_posts[site_name] = pd.DataFrame(data)
        print(f'{file_name} unzipped, containing {len(all_posts[site_name])} records')

In [3]:
# checking for subfolder, if not create
if not os.path.exists(f"./Data/comments/csv/"):
    os.makedirs("./Data/comments/csv") 
if not os.path.exists(f"./Data/posts/csv/"):
    os.makedirs("./Data/posts/csv") 
# exporting CSV
for i in all_comments.keys():
    all_comments[i].to_csv(f'./Data/comments/csv/{i}_comments.csv', index=False)
    print(f'{i} saved')
    
for i in all_posts.keys():
    all_posts[i].to_csv(f'./Data/posts/csv/{i.lower()}_posts.csv', index=False)
    print(f'{i} saved')

<a class="anchor" name="csv-imports"></a>

### Importing Files
#### Run from here if `csv` files are locally available

In [4]:
for file in os.listdir('./Data/comments/csv'):
    file_name = file[:-4]
    site_name = file.split('_')[0]
    all_comments[site_name] = pd.read_csv(f'./Data/comments/csv/{file}', low_memory=False)
    
for file in os.listdir('./Data/posts/csv'):
    file_name = file[:-4]
    site_name = file.split('_')[0]
    all_posts[site_name] = pd.read_csv(f'./Data/posts/csv/{file}', low_memory=False)

<a class="anchor" name="initial-cleaning"></a>

### Initial Cleaning
- filling nulls with blanks (numeric data later filled with `-1` in [datatype management](#datatype-management))
- dropped `removed` and `deleted` records
- dropped full-row duplicates and text body duplicates*
- generated metadata  
   
_*for this analysis, the content is more important than time of post and frequency. For other areas of analysis, consider leaving in text body duplicates._

In [5]:
# print('-----comments-----')
for site, dirty_file in all_comments.items():
    columns = [i for i in comments_cols if i in dirty_file.columns]
    file = dirty_file[columns].copy()
    removed_records = file[
        (file['body']=='[deleted]')|
        (file['body']=='[removed]')|
        (file['body'].str.contains('comment.+removed'))].index
    
    file.drop(index=removed_records, inplace=True)
    file.drop_duplicates(inplace=True)
    file.drop_duplicates(subset=['body'], inplace=True)
    original_length, new_length, unique_posts = len(dirty_file), len(file), len(file['parent_id'].unique())

    comments_cleaned[site] = file
    comments_full.append(original_length)
    comments_unique.append(new_length)
    comments_pct_removed.append((original_length-new_length)/original_length)
    unique_linked_posts.append(unique_posts)

    print(f'''{site}
    {original_length:,} records to {new_length:,} with {unique_posts:,} linked posts.
    {original_length-new_length:,} duplicate comments removed.''')
    if [i for i in comments_cols if i not in file.columns]:
        print(f'    columns not available: {[i for i in comments_cols if i not in file.columns]}')

depop
    810,794 records to 718,825 with 378,672 linked posts.
    91,969 duplicate comments removed.
etsysellers
    339,569 records to 310,249 with 162,599 linked posts.
    29,320 duplicate comments removed.
etsy
    662,752 records to 603,159 with 303,311 linked posts.
    59,593 duplicate comments removed.
flipping
    1,591,386 records to 1,475,464 with 773,985 linked posts.
    115,922 duplicate comments removed.
grailed
    55,955 records to 43,363 with 28,159 linked posts.
    12,592 duplicate comments removed.
poshmark
    566,272 records to 467,946 with 227,477 linked posts.
    98,326 duplicate comments removed.
stockx
    140,024 records to 126,593 with 75,929 linked posts.
    13,431 duplicate comments removed.
    columns not available: ['ups']


In [6]:
# print('-----posts-----')
for site, dirty_file in all_posts.items():
    columns = [i for i in posts_cols if i in dirty_file.columns]
    file = dirty_file[columns].copy()
    removed_records = file[
        (file['selftext']=='[deleted]')|
        (file['selftext']=='[removed]')|
        (file['selftext'].str.contains('post.+removed'))].index
    
    file.drop(index=removed_records, inplace=True)
    file.drop_duplicates(inplace=True)
    file.drop_duplicates(subset=['selftext'], inplace=True)
    original_length, new_length = len(dirty_file), len(file)
        
    posts_cleaned[site] = file
    posts_full.append(original_length)
    posts_unique.append(new_length)
    posts_pct_removed.append((original_length-new_length)/original_length)
    
    print(f'''{site}
    {original_length:,} records to {new_length:,}.
    {original_length-new_length:,} duplicate posts removed.''')
    if [i for i in posts_cols if i not in file.columns]:
        print(f'    columns not available: {[i for i in posts_cols if i not in file.columns]}')

depop
    126,523 records to 39,167.
    87,356 duplicate posts removed.
etsysellers
    38,292 records to 24,978.
    13,314 duplicate posts removed.
etsy
    98,967 records to 43,270.
    55,697 duplicate posts removed.
    columns not available: ['poll_data']
flipping
    92,907 records to 42,310.
    50,597 duplicate posts removed.
grailed
    20,449 records to 4,881.
    15,568 duplicate posts removed.
poshmark
    57,893 records to 22,881.
    35,012 duplicate posts removed.
stockx
    23,288 records to 10,321.
    12,967 duplicate posts removed.


In [7]:
comments_log = pd.DataFrame(zip(all_comments.keys(), 
                                comments_full, 
                                comments_unique,
                                unique_linked_posts,
                                comments_pct_removed), 
                            columns=['site','total records','unique records','unique linked posts','pct removed'])
# print(f"On average, {round(comments_log['pct removed'].mean(),4)*100}% of records removed")
# comments_log

In [8]:
posts_log = pd.DataFrame(zip(all_comments.keys(), 
                             posts_full, 
                             posts_unique, 
                             posts_pct_removed), 
                         columns=['site','total records','unique records','pct removed'])
# print(f"On average, {round(posts_log['pct removed'].mean(),4)*100}% of records removed")
# posts_log

<a class="anchor" name="datatype-management"></a>

### Setting and converting datatypes
#### Posts
- Convert numeric columns to `int` & fill nulls with `-1`: `created_utc`, `num_comments`, `score`, `ups`
- Convert binary to `bool`: `is_video`
- Generate timestamp `created` using `created_utc`

#### Comments
- Convert numeric columns to `int` & fill nulls with `-1`: `created_utc`, `score`, `ups`, `likes`
- Convert binary to `bool`: `controversiality`
- Generate timestamp `created` using `created_utc`

In [9]:
for site, file in posts_cleaned.items():
    for column, type in posts_cols_convert.items():
        file[column] = file[column].fillna(-1).astype(type)    
    file['created'] = file['created_utc'].apply(datetime.fromtimestamp)
    
for site, file in comments_cleaned.items():
    for column, type in {key: value for key, value in comments_cols_convert.items() if key in file.columns}.items():
        file[column] = file[column].fillna(-1).astype(type)   
    file['created'] = file['created_utc'].apply(datetime.fromtimestamp)

In [10]:
# comments_cleaned['depop'].dtypes

In [11]:
for file in posts_cleaned.values():
    start_dates.append(datetime.date(file['created'].min()))
    end_dates.append(datetime.date(file['created'].max()))

In [12]:
# checking for subfolder, if not create
if not os.path.exists(f"./Data/comments/cleaned/"):
    os.makedirs("./Data/comments/cleaned/") 
if not os.path.exists(f"./Data/posts/cleaned/"):
    os.makedirs("./Data/posts/cleaned/") 
# exporting CSV
for site, file in comments_cleaned.items():
    file.to_csv(f'./Data/comments/cleaned/{site}_comments.csv', index=False)
    print(f'{site}-cleaned saved')
    
for site, file in posts_cleaned.items():
    file.to_csv(f'./Data/posts/cleaned/{site}_posts.csv', index=False)
    print(f'{site}-cleaned saved')

depop-cleaned saved
etsysellers-cleaned saved
etsy-cleaned saved
flipping-cleaned saved
grailed-cleaned saved
poshmark-cleaned saved
stockx-cleaned saved
depop-cleaned saved
etsysellers-cleaned saved
etsy-cleaned saved
flipping-cleaned saved
grailed-cleaned saved
poshmark-cleaned saved
stockx-cleaned saved


In [13]:
posts_log['start date'] = start_dates
posts_log['end date'] = end_dates
# posts_log

In [14]:
if not os.path.exists(f"./Data/stats/"):
    os.makedirs("./Data/stats/") 

comments_log.to_csv('./Data/stats/comments.csv', index=False)
posts_log.to_csv('./Data/stats/posts.csv', index=False)